# Concat-net simple

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
from time import time
import os
import gzip
import numpy as np
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [ ]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [ ]:
X_train, y_train = load_mnist('drive/TCC-ITAU/fashion-mnist/data/fashion', kind='train')
X_test, y_test = load_mnist('drive/TCC-ITAU/fashion-mnist/data/fashion', kind='t10k')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1974)

In [ ]:
train_features = X_train.reshape(X_train.shape[0], 28, 28, 1)/255
vali_features = X_val.reshape(X_val.shape[0], 28, 28, 1)/255
test_features = X_test.reshape(X_test.shape[0], 28, 28, 1)/255

y_train_cat = np_utils.to_categorical(y_train)
y_val_cat = np_utils.to_categorical(y_val)
y_test_cat = np_utils.to_categorical(y_test)

## Coarse and Fine classe

| Label | Fine | Coarse |
| --- | --- | --- |
| 0 | T-shirt/top | TOP |
| 1 | Trouser | BOTTOM |
| 2 | Pullover | TOP |
| 3 | Dress |BODY |
| 4 | Coat | TOP |
| 5 | Sandal | FEET |
| 6 | Shirt | TOP |
| 7 | Sneaker | FEET |
| 8 | Bag | ACCESSORY |
| 9 | Ankle boot | FEET |

In [ ]:
dict_coarse = {0: 0,
              1: 1,
              2: 0,
              3: 2,
              4: 0,
              5: 3,
              6: 0,
              7: 3,
              8: 4,
              9: 3}

y_train_coarse = np.vectorize(dict_coarse.get)(y_train)
y_val_coarse = np.vectorize(dict_coarse.get)(y_val)
y_test_coarse = np.vectorize(dict_coarse.get)(y_test)

y_train_c_cat = np_utils.to_categorical(y_train_coarse)
y_val_c_cat = np_utils.to_categorical(y_val_coarse)
y_test_c_cat= np_utils.to_categorical(y_test_coarse)

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
from keras import optimizers
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras import backend as K

In [ ]:
class LossWeightsModifier(keras.callbacks.Callback):
  def __init__(self, alpha, beta):
    self.alpha = alpha
    self.beta = beta
    
  def on_epoch_end(self, epoch, logs={}):
    if epoch == 7:
      K.set_value(self.alpha, 0.6)
      K.set_value(self.beta, 0.4)
      print('Changing loss weights to: coarse = {}, fine = {}'.format(K.eval(self.alpha), K.eval(self.beta)))

    if epoch == 11:
      K.set_value(self.alpha, 0.2)
      K.set_value(self.beta, 0.8)
      print('Changing loss weights to: coarse = {}, fine = {}'.format(K.eval(self.alpha), K.eval(self.beta)))
      
    if epoch == 21:
      K.set_value(self.alpha, 0)
      K.set_value(self.beta, 1)
      print('Changing loss weights to: coarse = {}, fine = {}'.format(K.eval(self.alpha), K.eval(self.beta)))
      
    logs['alpha'] = K.eval(self.alpha)
    logs['beta'] = K.eval(self.beta)   

In [ ]:
img_rows, img_cols = 28, 28
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [ ]:
num_classes_coarse = 5
num_classes_fine = 10

img_input = Input(shape=input_shape, name='input')


x = Conv2D(32, (3, 3), activation='relu', name='block1_conv1')(img_input)
x = MaxPooling2D((2, 2), name='block1_pool')(x)

#---- coarse branch ----
coarse = Flatten(name='c1_flatten')(x)
coarse = Dense(64, name='c1_fc_1')(coarse)
coarse = BatchNormalization()(coarse)
coarse = Activation('relu')(coarse)
coarse = Dropout(0.5)(coarse)
coarse_pred = Dense(num_classes_coarse, activation='softmax', name='predictions_coarse')(coarse)

x = Conv2D(64, (3, 3), activation='relu', name='block1_conv2')(x)
x = MaxPooling2D((2, 2), name='block2_pool')(x)

x = Flatten(name='flatten')(x)
x = Dense(128, name='fc_1')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = keras.layers.concatenate([x, coarse])
fine_pred = Dense(num_classes_fine, activation='softmax', name='predictions_fine')(x)

In [ ]:
model = Model(inputs=img_input, outputs=[coarse_pred, fine_pred], name='concat_net')

In [20]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 26, 26, 32)   320         input[0][0]                      
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 13, 13, 32)   0           block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 11, 11, 64)   18496       block1_pool[0][0]                
__________________________________________________________________________________________________
block2_poo

In [ ]:
alpha = K.variable(value=0.95, dtype="float32", name="alpha") # A1 in paper
beta = K.variable(value=0.05, dtype="float32", name="beta") # A2 in paper

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              loss_weights=[alpha, beta],
              metrics=['accuracy'])

In [ ]:
class TimingCallback(keras.callbacks.Callback):
    
  def on_epoch_begin(self, epoch, logs={}):
    self.starttime=time()
  def on_epoch_end(self, epoch, logs={}):
    logs['time_elapsed'] = (time()-self.starttime)

In [ ]:
tb_cb = TensorBoard(log_dir='drive/TCC-ITAU/fashion-mnist/tensorboard/concat-net-simple', histogram_freq=0)
clr_cb = ReduceLROnPlateau(monitor='val_predictions_fine_loss', factor=0.5, patience=10, verbose=1)
time_cb = TimingCallback()
csv_cb = CSVLogger('drive/TCC-ITAU/fashion-mnist/training-data/concat-net/concat-net-simple.csv', separator=',', append=False)
change_lw = LossWeightsModifier(alpha, beta)

cbks = [tb_cb, clr_cb, time_cb, change_lw, csv_cb]

In [24]:
epochs = 50
start_time = time()
training = model.fit(train_features, [y_train_c_cat, y_train_cat],
                     validation_data=(vali_features, [y_val_c_cat, y_val_cat]),
                     epochs=epochs,
                     batch_size=512, 
                     callbacks=cbks,
                     verbose=1)
training_time = time() - start_time

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 4s 87us/step - loss: 0.2743 - predictions_coarse_loss: 0.2538 - predictions_fine_loss: 0.6639 - predictions_coarse_acc: 0.9269 - predictions_fine_acc: 0.7637 - val_loss: 0.1500 - val_predictions_coarse_loss: 0.1347 - val_predictions_fine_loss: 0.4414 - val_predictions_coarse_acc: 0.9630 - val_predictions_fine_acc: 0.8382
Epoch 2/50
48000/48000 [==============================] - 3s 70us/step - loss: 0.1274 - predictions_coarse_loss: 0.1139 - predictions_fine_loss: 0.3833 - predictions_coarse_acc: 0.9677 - predictions_fine_acc: 0.8627 - val_loss: 0.1009 - val_predictions_coarse_loss: 0.0887 - val_predictions_fine_loss: 0.3338 - val_predictions_coarse_acc: 0.9717 - val_predictions_fine_acc: 0.8788
Epoch 3/50
48000/48000 [==============================] - 3s 70us/step - loss: 0.1016 - predictions_coarse_loss: 0.0901 - predictions_fine_loss: 0.3197 - predictions_coarse_acc: 0.9722 - p

In [25]:
model.evaluate(test_features, [y_test_c_cat, y_test_cat])

10000/10000 [==============================] - 1s 145us/step


[0.37115275562927125, 0.08088710899380967, 0.37115275562927125, 0.9768, 0.9211]

In [26]:
model.save('drive/TCC-ITAU/fashion-mnist/models/concat-net-simple.h5')

TypeError: ignored

In [27]:
training_time

177.90673756599426